# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 03:27:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   61C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2018년 LG Data**

In [5]:
lexicon_2018 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2018_lg.csv')

lexicon_2018.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,LG화학,51910,매일경제,2018010118,中정부 전기차 면세 연장에 코스모화학 등 수혜株 주목,http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n 중국발 수요 증가에 힘입어 새해 '전기차 랠리'에 대한 기대감이 확산되...,2018-01-02,18,407000,415000,406500,411000,153739,0.014815,1,0,중국 수요 증가 새해 전기차 랠리 대한 대감 확산 한국 거래소 전기차 배터리 핵심 ...


In [6]:
lexicon_2018[lexicon_2018['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2018.dropna(axis=0, inplace=True)
lexicon_2018 = lexicon_2018.reset_index(drop=True)
lexicon_2018.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2018 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2018['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2018['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2018['Tokenization'][x].split())):
                    if lexicon_2018['Tokenization'][x].split()[w] == list(set(lexicon_2018['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2018['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2018['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2018 = pos_lexicon_2018.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2018['date'][x], positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z])))

***2018-01-02 Cosine Similarity between <중요한> & <가장> : 0.5371254086494446
***2018-01-02 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2018-01-02 Cosine Similarity between <적극적으로> & <적극> : 0.6639531850814819
***2018-01-02 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2018-01-02 Cosine Similarity between <증대> & <확보> : 0.5161436796188354
***2018-01-02 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2018-01-02 Cosine Similarity between <추진> & <추진> : 0.9999999403953552
***2018-01-02 Cosine Similarity between <추진> & <중인> : 0.539746880531311
***2018-01-02 Cosine Similarity between <추진> & <진행> : 0.5358568429946899
***2018-01-02 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2018-01-02 Cosine Similarity between <가치> & <의미> : 0.5354709625244141
***2018-01-02 Cosine Similarity between <우세> & <강세> : 0.5120405554771423
***2018-01-02 Cosine Similarity between <적극적으로> & <적극> : 0.6639531850814819
***2018-01-02 Cosine Similarity between <증대> 

In [11]:
pos_lexicon_2018

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2018-01-02,1,중요한,가장,0.537125,1
1,2018-01-02,1,증대,증가,0.694643,1
2,2018-01-02,2,적극적으로,적극,0.663953,1
3,2018-01-02,2,증대,증가,0.694643,1
4,2018-01-02,2,증대,확보,0.516144,1
...,...,...,...,...,...,...
8317,2019-01-02,1709,추진,계획,0.520542,2
8318,2019-01-02,1709,추진,사업,0.521374,4
8319,2019-01-02,1709,해결,문제,0.618619,2
8320,2019-01-02,1709,확보,확충,0.535100,1


In [12]:
pos_lexicon_2018.to_csv('../../../../Code/Data/Test/Stock-Year/pos_lg_2018.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2018['news_word'])))

a

['불황',
 '여의도고',
 '배규웅',
 '물량감소폭',
 '캔서롭캔서롭',
 '방법',
 '본격',
 '엠에스',
 '중요',
 '해법',
 '로수젯',
 '공영은',
 '매렸다',
 '호유에너지',
 '차익',
 '듀비메트',
 '확충',
 '피엠씨텍',
 '단점',
 '가장',
 '프럼파스트',
 '추진',
 '향상',
 '박진화',
 '스코폴레틴',
 '롱숏펀드',
 '티플랙스',
 '창저우시',
 '증설량',
 '남경시',
 '극복',
 '여준호',
 '활기',
 '드래콘',
 '확립',
 '꼽으셨는데',
 '너뿐이',
 '웃돈',
 '프로그램매매',
 '맥쿼리인프라',
 '고취',
 '전개',
 '언택트',
 '밑돈',
 '팩토바',
 '보락보락',
 '급락',
 '오혁수',
 '구글벤처스',
 '트라젠타',
 '방안',
 '물량감소',
 '씽크빅웅진씽크빅',
 '정찬',
 '검토',
 '재생기술',
 '강세',
 '침체',
 '프럼파스트프',
 '상쇄하',
 '들보',
 '실패',
 '증가',
 '매경테스트',
 '최고급',
 '보통신',
 '불스홀',
 '최고점',
 '중인',
 '듀비에',
 '래깅효',
 '증대',
 '구현',
 '애블린',
 '코스포켐텍',
 '조호윤',
 '장기물량',
 '발전',
 '랩스커버리',
 '의미',
 '적극',
 '웃돌',
 '촉진',
 '훈풍',
 '후이저우시',
 '니케이지수',
 '중거리핵전력조약',
 '요구',
 '이점',
 '임지아',
 '증진',
 '문제해결',
 '제미메트',
 '합천창녕보',
 '특성',
 '강력',
 '횡보할',
 '밑돈다',
 '꼽으셨어',
 '확보',
 '광풍',
 '기가와트',
 '카탕카',
 '인플렉트',
 '삼우엠스',
 '업황',
 '우세',
 '루브리컨츠',
 '판매승인',
 '팽팽하',
 '약세',
 '급증',
 '올릭스올릭스',
 '루텍재영솔루텍',
 '틸메르캅탄',
 '저하',
 '퍼부으',
 '강점',
 '나노메딕스',
 '파리올림픽',
 '시너지',
 '

In [14]:
b = list(set(list(pos_lexicon_2018[pos_lexicon_2018['cosine_similarity']>=0.7]['news_word'])))

b

['불황', '밑돈', '단점', '우세', '실패', '약세', '추진', '추천', '최고점']

In [15]:
c = list(set(list(pos_lexicon_2018[pos_lexicon_2018['cosine_similarity']>=0.65]['news_word'])))

c

['불황',
 '슈글렛',
 '밑돈',
 '적극',
 '웃돌',
 '단점',
 '우세',
 '실패',
 '증가',
 '물량감소',
 '약세',
 '감소',
 '추진',
 '추천',
 '최고점']